In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [5]:
df=pd.read_csv('geo_markers.csv')
df.head()

,DR_NO,AREA,AREA NAME,Rpt Dist No,LOCATION,Cross Street,LAT,LON
0,190326475.0,7.0,NaN,784.0,1900 S LONGWOOD AV,NaN,NaN,NaN
1,200106753.0,1.0,Central,182.0,NaN,NaN,34.0444,-118.2628
2,200320258.0,3.0,Southwest,356.0,1400 W 37TH ST,NaN,34.0210,-118.3002
3,NaN,9.0,NaN,964.0,14000 RIVERSIDE DR,NaN,NaN,NaN
4,200412582.0,4.0,Hollenbeck,413.0,200 E AVENUE 28,NaN,34.0820,NaN


In [6]:
df.isnull().sum()

,0
DR_NO,21091
AREA,21035
AREA NAME,21182
Rpt Dist No,21082
LOCATION,20765
Cross Street,60668
LAT,20960
LON,21022


In [7]:
df.drop("DR_NO",axis=1,inplace=True)

In [8]:
df.head(10)

,AREA,AREA NAME,Rpt Dist No,LOCATION,Cross Street,LAT,LON
0,7.0,NaN,784.0,1900 S LONGWOOD AV,NaN,NaN,NaN
1,1.0,Central,182.0,NaN,NaN,34.0444,-118.2628
2,3.0,Southwest,356.0,1400 W 37TH ST,NaN,34.0210,-118.3002
3,9.0,NaN,964.0,14000 RIVERSIDE DR,NaN,NaN,NaN
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,NaN,34.0820,NaN
5,2.0,Rampart,NaN,NaN,NaN,34.0642,NaN
6,2.0,Rampart,NaN,NaN,NaN,34.0536,-118.2788
7,13.0,Newton,1333.0,NaN,NaN,34.0170,-118.2643
8,11.0,Northeast,1161.0,KENMORE ST,FOUNTAIN,34.0953,-118.2974
9,1.0,Central,NaN,NaN,NaN,34.0710,NaN


In [9]:
# 2. Fill from AREA centroid
# -------------------------------
# Compute mean lat/lon per AREA
area_centroids = df.groupby('AREA')[['LAT', 'LON']].mean()

# Fill missing LAT/LON with AREA centroid
for idx, row in df[df[['LAT','LON']].isna().any(axis=1)].iterrows():
    area = row['AREA']
    if pd.notna(area) and area in area_centroids.index:
        if pd.isna(row['LAT']):
            df.at[idx, 'LAT'] = area_centroids.loc[area, 'LAT']
        if pd.isna(row['LON']):
            df.at[idx, 'LON'] = area_centroids.loc[area, 'LON']

In [10]:
df.isnull().sum()

,0
AREA,21035
AREA NAME,21182
Rpt Dist No,21082
LOCATION,20765
Cross Street,60668
LAT,6222
LON,6311


In [11]:
knn_imputer = KNNImputer(n_neighbors=5)
df[['LAT','LON']] = knn_imputer.fit_transform(df[['LAT','LON']])

In [12]:
df.isnull().sum()

,0
AREA,21035
AREA NAME,21182
Rpt Dist No,21082
LOCATION,20765
Cross Street,60668
LAT,0
LON,0


In [13]:
df['AREA'] = df['AREA'].fillna(df['AREA'].median())
df['Rpt Dist No'] = df['Rpt Dist No'].fillna(df['Rpt Dist No'].median())

In [14]:
df.isnull().sum()

,0
AREA,0
AREA NAME,21182
Rpt Dist No,0
LOCATION,20765
Cross Street,60668
LAT,0
LON,0


In [15]:
df.drop("Cross Street",axis=1,inplace=True)

In [16]:
df.head()

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON
0,7.0,NaN,784.0,1900 S LONGWOOD AV,33.958997,-117.862377
1,1.0,Central,182.0,NaN,34.044400,-118.262800
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200
3,9.0,NaN,964.0,14000 RIVERSIDE DR,34.099159,-118.259943
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643


In [17]:
# Create AREA → AREA NAME mapping from non-missing rows
area_to_name = df.dropna(subset=['AREA', 'AREA NAME']).drop_duplicates('AREA').set_index('AREA')['AREA NAME']

# Fill AREA NAME from AREA mapping
df['AREA NAME'] = df['AREA NAME'].fillna(df['AREA'].map(area_to_name))


In [18]:
df.isnull().sum()

,0
AREA,0
AREA NAME,0
Rpt Dist No,0
LOCATION,20765
LAT,0
LON,0


In [19]:
df.head()

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON
0,7.0,77th Street,784.0,1900 S LONGWOOD AV,33.958997,-117.862377
1,1.0,Central,182.0,NaN,34.044400,-118.262800
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200
3,9.0,Van Nuys,964.0,14000 RIVERSIDE DR,34.099159,-118.259943
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643


In [20]:
df['LOCATION'] = df.groupby('AREA')['LOCATION'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))


/tmp/ipython-input-1397773040.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['LOCATION'] = df.groupby('AREA')['LOCATION'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))


In [21]:
df.isnull().sum()

,0
AREA,0
AREA NAME,0
Rpt Dist No,0
LOCATION,1
LAT,0
LON,0


In [22]:
df.head()

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON
0,7.0,77th Street,784.0,1900 S LONGWOOD AV,33.958997,-117.862377
1,1.0,Central,182.0,800 N ALAMEDA ST,34.044400,-118.262800
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200
3,9.0,Van Nuys,964.0,14000 RIVERSIDE DR,34.099159,-118.259943
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643


In [23]:
df1=pd.read_csv('crime_blueprint.csv')
df1.head()

,DR_NO,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4
0,190326475.0,1.0,NaN,VEHICLE - STOLEN,510.0,998.0,NaN,NaN
1,200106753.0,1.0,330.0,BURGLARY FROM VEHICLE,NaN,998.0,NaN,NaN
2,200320258.0,1.0,480.0,BIKE - STOLEN,480.0,NaN,NaN,NaN
3,NaN,1.0,NaN,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.0,NaN,NaN,NaN
4,200412582.0,1.0,510.0,VEHICLE - STOLEN,510.0,NaN,NaN,NaN


In [24]:
df1= df1.drop(columns=['DR_NO'], errors='ignore')
df1= df1.drop(columns=['Crm Cd 3'], errors='ignore')
df1= df1.drop(columns=['Crm Cd 4'], errors='ignore')

In [25]:
df1.head()

,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,Crm Cd 2
0,1.0,NaN,VEHICLE - STOLEN,510.0,998.0
1,1.0,330.0,BURGLARY FROM VEHICLE,NaN,998.0
2,1.0,480.0,BIKE - STOLEN,480.0,NaN
3,1.0,NaN,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.0,NaN
4,1.0,510.0,VEHICLE - STOLEN,510.0,NaN


In [26]:
df1['Part 1-2'].value_counts()

,count
Part 1-2,
1.0,28685
2.0,20280


In [27]:
# Step 1: Group-by imputation
df1['Part 1-2'] = df1.groupby('Crm Cd')['Part 1-2']\
                   .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

# Step 2: Fill any remaining nulls with global mode
df1['Part 1-2'].fillna(df1['Part 1-2'].mode()[0], inplace=True)

/tmp/ipython-input-133552894.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Part 1-2'].fillna(df1['Part 1-2'].mode()[0], inplace=True)


In [28]:
df1.isnull().sum()

,0
Part 1-2,0
Crm Cd,21182
Crm Cd Desc,21082
Crm Cd 1,20767
Crm Cd 2,66060


In [29]:
df1['Part 1-2'].value_counts()

,count
Part 1-2,
1.0,49738
2.0,20262


In [30]:
# Keep only rows with 1.0 or 2.0 in 'Part 1-2'
df_bin = df1[df1['Part 1-2'].isin([1.0, 2.0])].copy()

class_1 = df_bin[df_bin['Part 1-2'] == 1.0]
class_2 = df_bin[df_bin['Part 1-2'] == 2.0]

total = len(df_bin)
target_1 = int(total * 0.60)           # desired count for class 1
target_2 = total - target_1           # desired count for class 2 (ensures exact 60:40)

# Undersample class 1 (without replacement) if needed
if len(class_1) > target_1:
    class_1_resampled = class_1.sample(n=target_1, replace=False, random_state=42)
else:
    # Cannot undersample to increase size; keep as-is (or optionally oversample instead)
    class_1_resampled = class_1.copy()
    target_1 = len(class_1_resampled)
    target_2 = total - target_1  # adjust target_2 accordingly

# Oversample class 2 (with replacement if needed)
if len(class_2) >= target_2:
    class_2_resampled = class_2.sample(n=target_2, replace=False, random_state=42)
else:
    class_2_resampled = class_2.sample(n=target_2, replace=True, random_state=42)

# Combine and shuffle
df_balanced = pd.concat([class_1_resampled, class_2_resampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Check final distribution
print(df_balanced['Part 1-2'].value_counts(), '\n')
print((df_balanced['Part 1-2'].value_counts(normalize=True) * 100).round(4))

Part 1-2
1.0    42000
2.0    28000
Name: count, dtype: int64 

Part 1-2
1.0    60.0
2.0    40.0
Name: proportion, dtype: float64


In [31]:
df1.head()

,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,Crm Cd 2
0,1.0,NaN,VEHICLE - STOLEN,510.0,998.0
1,1.0,330.0,BURGLARY FROM VEHICLE,NaN,998.0
2,1.0,480.0,BIKE - STOLEN,480.0,NaN
3,1.0,NaN,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.0,NaN
4,1.0,510.0,VEHICLE - STOLEN,510.0,NaN


In [32]:
from sklearn.impute import KNNImputer

# Columns for imputation
cols_for_knn = ['Crm Cd', 'Crm Cd 1', 'Crm Cd 2']

# Sample subset for fitting KNN
sample_df = df1[cols_for_knn].sample(n=5000, random_state=42)  # reduce to 5k rows
imputer = KNNImputer(n_neighbors=5)

# Fit on subset
imputer.fit(sample_df)

# Transform full dataset
df1[cols_for_knn] = imputer.transform(df1[cols_for_knn])

In [33]:
df1.isnull().sum()

,0
Part 1-2,0
Crm Cd,0
Crm Cd Desc,21082
Crm Cd 1,0
Crm Cd 2,0


In [34]:
code_to_desc = df1.dropna(subset=['Crm Cd', 'Crm Cd Desc'])\
                 .drop_duplicates(subset=['Crm Cd'])\
                 .set_index('Crm Cd')['Crm Cd Desc']
df1['Crm Cd Desc'] = df1.apply(
    lambda row: code_to_desc.get(row['Crm Cd'], row['Crm Cd Desc']),
    axis=1
)
df1['Crm Cd Desc'].fillna(df1['Crm Cd Desc'].mode()[0], inplace=True)

/tmp/ipython-input-1459363831.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['Crm Cd Desc'].fillna(df1['Crm Cd Desc'].mode()[0], inplace=True)


In [35]:
df1.isnull().sum()

,0
Part 1-2,0
Crm Cd,0
Crm Cd Desc,0
Crm Cd 1,0
Crm Cd 2,0


In [36]:
combined_df=pd.concat([df,df1],axis=1)
combined_df

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,Crm Cd 2
0,7.0,77th Street,784.0,1900 S LONGWOOD AV,33.958997,-117.862377,1.0,474.00000,VEHICLE - STOLEN,510.000000,998.000000
1,1.0,Central,182.0,800 N ALAMEDA ST,34.044400,-118.262800,1.0,330.00000,BURGLARY FROM VEHICLE,330.000000,998.000000
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200,1.0,480.00000,BIKE - STOLEN,480.000000,929.000000
3,9.0,Van Nuys,964.0,14000 RIVERSIDE DR,34.099159,-118.259943,1.0,343.00000,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.000000,788.200000
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643,1.0,510.00000,VEHICLE - STOLEN,510.000000,998.000000
...,...,...,...,...,...,...,...,...,...,...,...
69995,13.0,Newton,1303.0,700 E 12TH ST,33.972428,-118.253300,1.0,230.00000,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",230.000000,236.000000
69996,7.0,77th Street,1269.0,100 THE GROVE DR,33.962900,-118.260800,1.0,502.51598,VEHICLE - STOLEN,506.322734,948.806338
69997,8.0,West LA,702.0,10900 ASHTON AV,34.057800,-118.443300,1.0,440.00000,THEFT PLAIN - PETTY ($950 & UNDER),440.000000,594.200000
69998,9.0,Van Nuys,943.0,5500 VAN NUYS BL,34.174700,-118.259943,1.0,442.00000,SHOPLIFTING - PETTY THEFT ($950 & UNDER),442.000000,812.400000


In [37]:
df2=pd.read_csv("chrono_trace.csv")
df2.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC
0,190326475.0,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130.0
1,NaN,02/09/2020 12:00:00 AM,NaN,1800.0
2,200320258.0,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,NaN
3,NaN,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037.0
4,200412582.0,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630.0


In [38]:
df2= df2.drop(columns=['DR_NO'], errors='ignore')
df2.head()

,Date Rptd,DATE OCC,TIME OCC
0,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130.0
1,02/09/2020 12:00:00 AM,NaN,1800.0
2,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,NaN
3,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037.0
4,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630.0


In [39]:
df2['TIME OCC'] = df2.groupby('DATE OCC')['TIME OCC']\
                     .transform(lambda x: x.fillna(x.median()))
# Fallback: fill any remaining with global median
#df2['TIME OCC'].fillna(df2['TIME OCC'].median(), inplace=True)


In [40]:
df2.isnull().sum()

,0
Date Rptd,20868
DATE OCC,20945
TIME OCC,20945


In [41]:
# Fill missing dates from Date Rptd if available
df2['DATE OCC'].fillna(df2['Date Rptd'], inplace=True)
# If still missing, fill with most frequent date
df2['DATE OCC'].fillna(df2['DATE OCC'].mode()[0], inplace=True)

/tmp/ipython-input-684880501.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['DATE OCC'].fillna(df2['Date Rptd'], inplace=True)


In [42]:
df2.isnull().sum()

,0
Date Rptd,20868
DATE OCC,0
TIME OCC,20945


In [43]:
df2['Date Rptd'].fillna(df2['DATE OCC'], inplace=True)
# Fallback to mode
df2['Date Rptd'].fillna(df2['Date Rptd'].mode()[0], inplace=True)


/tmp/ipython-input-302415293.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Date Rptd'].fillna(df2['DATE OCC'], inplace=True)


In [44]:
df2.isnull().sum()

,0
Date Rptd,0
DATE OCC,0
TIME OCC,20945


In [45]:
third_df=pd.concat([combined_df,df2],axis=1)
third_df.shape

(70000, 14)

In [46]:
df3=pd.read_csv("case_closure.csv")
df3.head()

,DR_NO,Status,Status Desc
0,190326475.0,AA,Adult Arrest
1,NaN,IC,Invest Cont
2,NaN,NaN,NaN
3,200907217.0,IC,Invest Cont
4,200412582.0,IC,Invest Cont


In [47]:
df3= df3.drop(columns=['DR_NO'], errors='ignore')
df3.head(10)

,Status,Status Desc
0,AA,Adult Arrest
1,IC,Invest Cont
2,NaN,NaN
3,IC,Invest Cont
4,IC,Invest Cont
5,NaN,NaN
6,IC,NaN
7,IC,NaN
8,AA,Adult Arrest
9,NaN,Invest Cont


In [48]:
# Create mapping from known pairs
mapping = df3.dropna().drop_duplicates(subset=["Status", "Status Desc"]) \
            .set_index("Status")["Status Desc"].to_dict()

# Fill Status based on Status Desc
reverse_mapping = {v: k for k, v in mapping.items()}

df3["Status"] = df3.apply(
    lambda row: reverse_mapping.get(row["Status Desc"], row["Status"])
    if pd.isna(row["Status"]) else row["Status"], axis=1
)

# Fill Status Desc based on Status
df3["Status Desc"] = df3.apply(
    lambda row: mapping.get(row["Status"], row["Status Desc"])
    if pd.isna(row["Status Desc"]) else row["Status Desc"], axis=1
)

In [49]:
df3.isnull().sum()

,0
Status,4640
Status Desc,4640


In [50]:
df3[["Status", "Status Desc"]] = df3[["Status", "Status Desc"]].apply(lambda col: col.fillna(col.mode()[0]))

In [51]:
df3.isnull().sum()

,0
Status,0
Status Desc,0


In [52]:
fourth_df=pd.concat([third_df,df3],axis=1)
fourth_df.head()

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,Crm Cd 2,Date Rptd,DATE OCC,TIME OCC,Status,Status Desc
0,7.0,77th Street,784.0,1900 S LONGWOOD AV,33.958997,-117.862377,1.0,474.0,VEHICLE - STOLEN,510.0,998.0,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130.0,AA,Adult Arrest
1,1.0,Central,182.0,800 N ALAMEDA ST,34.044400,-118.262800,1.0,330.0,BURGLARY FROM VEHICLE,330.0,998.0,02/09/2020 12:00:00 AM,02/09/2020 12:00:00 AM,NaN,IC,Invest Cont
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200,1.0,480.0,BIKE - STOLEN,480.0,929.0,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1600.0,IC,Invest Cont
3,9.0,Van Nuys,964.0,14000 RIVERSIDE DR,34.099159,-118.259943,1.0,343.0,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.0,788.2,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037.0,IC,Invest Cont
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643,1.0,510.0,VEHICLE - STOLEN,510.0,998.0,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630.0,IC,Invest Cont


In [53]:
df4=pd.read_csv("misc_matrix.csv")
df4.head()

,DR_NO,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc
0,NaN,NaN,0.0,M,O,101.0,STREET,NaN,NaN
1,NaN,1822 1402 0344,47.0,NaN,O,NaN,NaN,NaN,NaN
2,NaN,NaN,19.0,NaN,NaN,502.0,NaN,NaN,NaN
3,200907217.0,0325 1501,NaN,NaN,O,405.0,CLOTHING STORE,NaN,NaN
4,NaN,NaN,0.0,NaN,NaN,NaN,STREET,NaN,NaN


In [54]:
df4= df4.drop(columns=['DR_NO'], errors='ignore')
df4.head()

,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc
0,NaN,0.0,M,O,101.0,STREET,NaN,NaN
1,1822 1402 0344,47.0,NaN,O,NaN,NaN,NaN,NaN
2,NaN,19.0,NaN,NaN,502.0,NaN,NaN,NaN
3,0325 1501,NaN,NaN,O,405.0,CLOTHING STORE,NaN,NaN
4,NaN,0.0,NaN,NaN,NaN,STREET,NaN,NaN


In [55]:
# Mapping-based imputation for Weapon columns
weapon_map = df4.dropna(subset=["Weapon Used Cd", "Weapon Desc"]) \
               .drop_duplicates(subset=["Weapon Used Cd", "Weapon Desc"]) \
               .set_index("Weapon Used Cd")["Weapon Desc"].to_dict()

# Fill Weapon Desc from Weapon Used Cd
df4["Weapon Desc"] = df4.apply(
    lambda row: weapon_map.get(row["Weapon Used Cd"], row["Weapon Desc"])
    if pd.isna(row["Weapon Desc"]) else row["Weapon Desc"], axis=1
)

# Fill Weapon Used Cd from Weapon Desc (reverse map)
reverse_weapon_map = {v: k for k, v in weapon_map.items()}
df4["Weapon Used Cd"] = df4.apply(
    lambda row: reverse_weapon_map.get(row["Weapon Desc"], row["Weapon Used Cd"])
    if pd.isna(row["Weapon Used Cd"]) else row["Weapon Used Cd"], axis=1
)

# Median for Vict Age
df4["Vict Age"] = df4["Vict Age"].fillna(df4["Vict Age"].median())

# Mode for categorical columns
for col in ["Weapon Used Cd", "Vict Descent", "Vict Sex", "Premis Cd", "Premis Desc","Mocodes"]:
    df4[col] = df4[col].fillna(df4[col].mode()[0])

In [56]:
df4.isnull().sum()

,0
Mocodes,0
Vict Age,0
Vict Sex,0
Vict Descent,0
Premis Cd,0
Premis Desc,0
Weapon Used Cd,0
Weapon Desc,45232


In [57]:
df4.head()

,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc
0,0344,0.0,M,O,101.0,STREET,400.0,NaN
1,1822 1402 0344,47.0,M,O,101.0,STREET,400.0,NaN
2,0344,19.0,M,H,502.0,STREET,400.0,NaN
3,0325 1501,31.0,M,O,405.0,CLOTHING STORE,400.0,NaN
4,0344,0.0,M,H,101.0,STREET,400.0,NaN


In [58]:
fifth_df=pd.concat([fourth_df,df4],axis=1)
fifth_df.head()

,AREA,AREA NAME,Rpt Dist No,LOCATION,LAT,LON,Part 1-2,Crm Cd,Crm Cd Desc,Crm Cd 1,...,Status,Status Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc
0,7.0,77th Street,784.0,1900 S LONGWOOD AV,33.958997,-117.862377,1.0,474.0,VEHICLE - STOLEN,510.0,...,AA,Adult Arrest,0344,0.0,M,O,101.0,STREET,400.0,NaN
1,1.0,Central,182.0,800 N ALAMEDA ST,34.044400,-118.262800,1.0,330.0,BURGLARY FROM VEHICLE,330.0,...,IC,Invest Cont,1822 1402 0344,47.0,M,O,101.0,STREET,400.0,NaN
2,3.0,Southwest,356.0,1400 W 37TH ST,34.021000,-118.300200,1.0,480.0,BIKE - STOLEN,480.0,...,IC,Invest Cont,0344,19.0,M,H,502.0,STREET,400.0,NaN
3,9.0,Van Nuys,964.0,14000 RIVERSIDE DR,34.099159,-118.259943,1.0,343.0,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),343.0,...,IC,Invest Cont,0325 1501,31.0,M,O,405.0,CLOTHING STORE,400.0,NaN
4,4.0,Hollenbeck,413.0,200 E AVENUE 28,34.082000,-117.673643,1.0,510.0,VEHICLE - STOLEN,510.0,...,IC,Invest Cont,0344,0.0,M,H,101.0,STREET,400.0,NaN


In [59]:
fifth_df[['Weapon Desc','Weapon Used Cd']].value_counts()

,,count
Weapon Desc,Weapon Used Cd,
"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400.0,13249
UNKNOWN WEAPON/OTHER WEAPON,500.0,2719
VERBAL THREAT,511.0,1836
HAND GUN,102.0,1457
KNIFE WITH BLADE 6INCHES OR LESS,200.0,533
...,...,...
ANTIQUE FIREARM,116.0,1
M1-1 SEMIAUTOMATIC ASSAULT RIFLE,123.0,1
BOWIE KNIFE,202.0,1


In [60]:
fifth_df.to_csv("Final_Data.csv",index=False)

/content/Final_Data.csv